In [1]:
%matplotlib inline
from parcels import FieldSet, Field, ParticleSet, AdvectionRK4, JITParticle, ErrorCode, VectorField, Variable
from datetime import timedelta as delta
from glob import glob
import numpy as np
import xarray as xr

In [4]:
ddir = '/data2/imau/oceanparcels/hydrodynamic_data/NEMO-MEDUSA/ORCA025-N006/'
ufiles = sorted(glob(ddir+'means/ORCA025-N06_*d05U.nc'))
vfiles = [u.replace('05U.nc', '05V.nc') for u in ufiles]
meshfile = glob(ddir+'domain/coordinates.nc')

nemofiles = {'U': {'lon': meshfile, 'lat': meshfile, 'data': ufiles},
             'V': {'lon': meshfile, 'lat': meshfile, 'data': vfiles},
             'Mask': {'lon': meshfile, 'lat': meshfile, 'data': ddir+'domain/mask.nc'}}
nemovariables = {'U': 'uo', 'V': 'vo', 'Mask': 'tmask'}
nemodimensions = {'lon': 'glamf', 'lat': 'gphif', 'time': 'time_counter'}
fieldset = FieldSet.from_nemo(nemofiles, nemovariables, nemodimensions)

fieldset.computeTimeChunk(0, 1)

864000

In [5]:
variables = {'Unemo_unbeach': 'unBeachU', 'Vnemo_unbeach': 'unBeachV'}
dimensions = {'lon': 'glamf', 'lat': 'gphif'}
# File created from https://github.com/OceanParcels/Parcelsv2.0PaperNorthSeaScripts/blob/master/preprocess/unbeach_Cfield.py
fieldsetUnBeach = FieldSet.from_nemo(ddir+'domain/ORCA*_unbeaching_vel.nc', variables, dimensions, tracer_interp_method='cgrid_velocity')
fieldset.add_field(fieldsetUnBeach.Unemo_unbeach)
fieldset.add_field(fieldsetUnBeach.Vnemo_unbeach)

UVunbeach = VectorField('UVunbeach', fieldset.Unemo_unbeach, fieldset.Vnemo_unbeach)
fieldset.add_vector_field(UVunbeach)

In [10]:
def UnBeaching(particle, fieldset, time):
    m = fieldset.Mask[0, particle.depth, particle.lat, particle.lon]
    if m < 0.99:
        (ub, vb) = fieldset.UVunbeach[time, particle.depth, particle.lat, particle.lon]
        particle.lon += ub * particle.dt
        particle.lat += vb * particle.dt

def DeleteParticle(particle, fieldset, time):
    particle.delete()

In [26]:
fieldset.check_complete()
lons = []
lats = []
for lon in np.arange(360-80, 350, 0.5):
    for lat in np.arange(10, 50, 0.5):
        if fieldset.Mask[0, 0, lat, lon] > 0.99:
            lons.append(lon)
            lats.append(lat)
print('%d particle locations' % len(lons))

9977 particle locations


In [28]:
for rmsdvel in [0, 5, 7, 10, 15, 20, 30, 99]:
    fname = "/scratch/evansebille/skimrmsdvel_asfield_%.2dcms.nc" % rmsdvel
    print(fname)

    pset = ParticleSet(fieldset=fieldset, pclass=JITParticle, lon=lons, lat=lats)
    fieldset.rmsdvel = rmsdvel/100.

    def compute(fieldset):
        # Adding random field
        for f in [fieldset.U, fieldset.V]:
            for tind in f.loaded_time_indices:
                f.data[tind, :] = f.data[tind, :] + np.random.normal(0, fieldset.rmsdvel, (f.data.shape[-2], f.data.shape[-1]))

    fieldset.compute_on_defer = compute

    outfile = pset.ParticleFile(name=fname, outputdt=delta(days=5))

    pset.execute(AdvectionRK4+pset.Kernel(UnBeaching), dt=delta(hours=1), output_file=outfile,
                 recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})
    outfile.close()

/scratch/evansebille/skimrmsdvel_asfield_20cms.nc


INFO: Compiled JITParticleAdvectionRK4UnBeaching ==> /tmp/parcels-24236/39731944c1853328bfe6edc3479f83d9.so
INFO: Temporary output files are stored in /scratch/evansebille/out-SFGBXWJD.
INFO: You can use "parcels_convert_npydir_to_netcdf /scratch/evansebille/out-SFGBXWJD" to convert these to a NetCDF file during the run.
100% (94262400.0 of 94262400.0) |########| Elapsed Time: 0:14:40 Time:  0:14:40


/scratch/evansebille/skimrmsdvel_asfield_30cms.nc


INFO: Compiled JITParticleAdvectionRK4UnBeaching ==> /tmp/parcels-24236/28b9ad32ffc2ff8a8b8a06edd7b3d90f.so
INFO: Temporary output files are stored in /scratch/evansebille/out-NEUEURAT.
INFO: You can use "parcels_convert_npydir_to_netcdf /scratch/evansebille/out-NEUEURAT" to convert these to a NetCDF file during the run.
100% (94262400.0 of 94262400.0) |########| Elapsed Time: 0:13:26 Time:  0:13:26
